### 数据集处理

首先，去除头部relation。

然后，获取train set中instance小于100的关系

最后，根据第二步的关系从test set中获取最终测试集合

In [ ]:
import pandas as pd
import json

# 1. 移除头部的relation

In [101]:
def remove_headRel(old_fp, new_fp):
    count = 0
    # 这个具体头部移除那些，根据relation的断层决定
    remove_rel_list = ['NA', 'per:title', 'org:top_members/employees', 'per:employee_of', 'org:alternate_names']
    with open(new_fp, 'w', encoding='utf-8') as f_op:
        with open(old_fp, 'r', encoding = 'utf-8') as f:
            for line in f.readlines():
                line = json.loads(line) # loads(字符串)， load(文件名字)
                if line['relation'] not in remove_rel_list:
                    json.dump(line, f_op)
                    f_op.write('\n')
                    count+=1
    return count

count1 = remove_headRel('tacred_test.txt', 'tacred_37_test')
count2 = remove_headRel('tacred_valid.txt', 'tacred_37_valid')
count3 = remove_headRel('tacred_train.txt', 'tacred_37_train')
print(count1, count2, count3)

2002 3270 6347


# 2. 从train set中找到instance小于100的关系

In [108]:
def get_relation_dict(file_path):
    rel_fre_dict = {}
    with open(file_path, 'r', encoding = 'utf-8') as f:
        # 2. 建立词频表
        for line in f.readlines():
            line = json.loads(line) # loads(字符串)， load(文件名字)
            if line['relation'] not in rel_fre_dict.keys():
                rel_fre_dict[line['relation']] = 1
            else:
                rel_fre_dict[line['relation']] += 1
    return rel_fre_dict

trainset_f = get_relation_dict('tacred_train.txt')
new_dict = {}
total = 0
for i in trainset_f.keys():
    if trainset_f[i]<100:
        new_dict[i] = trainset_f[i]
        total+=trainset_f[i]
print(len(new_dict),total, new_dict.keys())

13 720 dict_keys(['org:founded', 'per:date_of_birth', 'per:stateorprovince_of_birth', 'per:country_of_birth', 'per:city_of_birth', 'per:country_of_death', 'per:religion', 'org:shareholders', 'per:charges', 'org:number_of_employees/members', 'org:dissolved', 'per:city_of_death', 'per:stateorprovince_of_death'])


# 3. 根据步骤2中得到的关系，从test set中取出这些关系及其instances，作为Hits@K评测指标的测试集合，即实际使用的test set

In [106]:
train_100 = ['org:founded', 'per:date_of_birth', 'per:stateorprovince_of_birth', 'per:country_of_birth', 'per:city_of_birth', 'per:country_of_death', 'per:religion', 'org:shareholders', 'per:charges', 'org:number_of_employees/members', 'org:dissolved', 'per:city_of_death', 'per:stateorprovince_of_death']
def create_rel100(old_fp, new_fp, train_100):
    count = 0
    rel_fre_dict = {}
    remove_rel_list = ['NA', 'per:title', 'org:top_members/employees', 'per:employee_of', 'org:alternate_names']
    with open(new_fp, 'w', encoding='utf-8') as f_op:
        with open(old_fp, 'r', encoding = 'utf-8') as f:
            for line in f.readlines():
                line = json.loads(line) # loads(字符串)， load(文件名字)
                if line['relation'] in train_100:
                    json.dump(line, f_op)
                    f_op.write('\n')
                    count+=1
                    if line['relation'] not in rel_fre_dict.keys():
                        rel_fre_dict[line['relation']] = 1
                    else:
                        rel_fre_dict[line['relation']] += 1
                
    return count, rel_fre_dict
count, rel_fre = create_rel100('tacred_37_test', 'tacred_lessThan100_test', train_100)
print(count)
print(rel_fre)

299
{'org:founded': 37, 'per:date_of_birth': 9, 'per:charges': 103, 'per:country_of_birth': 5, 'per:city_of_birth': 5, 'per:country_of_death': 9, 'per:religion': 47, 'org:shareholders': 13, 'per:stateorprovince_of_birth': 8, 'org:number_of_employees/members': 19, 'org:dissolved': 2, 'per:city_of_death': 28, 'per:stateorprovince_of_death': 14}


# 4. 生成一个固定的集合存放meta-testing support和query的每个task

In [109]:
def task_generator_ForTestTask(dataset_path, Nways, Kshots):
    dataset = []
    tasks = []
    with open(dataset_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            line = eval(line)
            dataset.append(line)
    print(len(dataset))
    t = []
    for line in dataset:
        t.append(line)
        if len(t)/5 == 1:
            tasks.append(t)
            t = []
    return tasks

tasks = task_generator_ForTestTask('tacred_lessThan100_test', 5, 1)
print(len(tasks[0]))
print(tasks[0])
with open('tacred_37-100_test_query.txt', 'w', encoding = 'utf-8') as f:
    f.write(str(tasks))

299
5
[{'text': 'The National Congress of American Indians was founded in 1944 in response to assimilation policies being imposed on tribes by the federal government .', 'relation': 'org:founded', 'h': {'id': '0', 'name': 'National Congress of American Indians', 'pos': [4, 41]}, 't': {'id': '1', 'name': '1944', 'pos': [57, 61]}}, {'text': 'UASR was founded in 1989 by Mousa abu Mazook < http://wwwinvestigativeprojectorg/profile/106 > , who currently serves as the Deputy Chief of the HAMAS political bureau in Damascus , Syria , and has been listed as a Specially Designated Global Terrorist by the US government in 1995 .', 'relation': 'org:founded', 'h': {'id': '0', 'name': 'UASR', 'pos': [1, 4]}, 't': {'id': '1', 'name': '1989', 'pos': [20, 24]}}, {'text': 'Founded in 1951 , PATA is a non-profit membership association dedicated to building responsible development of the Asia Pacific travel and tourism industry .', 'relation': 'org:founded', 'h': {'id': '0', 'name': 'PATA', 'pos': [18, 22

In [111]:
import random
def task_generator_byTask(tasks,dataset_path):
    dataset = {}
    with open(dataset_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            line = eval(line)
            if line["relation"] in dataset.keys():
                dataset[line["relation"]].append(line)
            else:
                dataset[line["relation"]] = []
                dataset[line["relation"]].append(line)
    new_task = []   
    for task in tasks:
        rel = [i['relation'] for i in task]
        t = []
        for i, r in enumerate(rel):
            if len(dataset[r])>1:
                temp = random.sample(dataset[r], 1)
                while(temp == task[i]):
                    temp = random.sample(datase[r], 1)
                t.append(temp[0])
            else:
                temp = random.sample(dataset[r], 1)
                t.append(temp[0])
        new_task.append(t)
    print(len(new_task))
    return new_task

new_tasks = task_generator_byTask(tasks,'tacred_lessThan100_test')

with open('tacred_37-100_test_support.txt', 'w', encoding = 'utf-8') as f:
    f.write(str(new_tasks))

59


# 5. 根据调整后的集合生成rel2id的json文件

In [11]:
import json
remove_rel_list = ['NA', 'per:title', 'org:top_members/employees', 'per:employee_of', 'org:alternate_names']
def get_relation_dict(file_path):
    rel_fre_dict = {}
    with open(file_path, 'r', encoding = 'utf-8') as f:
        # 2. 建立词频表
        for line in f.readlines():
            line = json.loads(line) # loads(字符串)， load(文件名字)
            if line['relation'] not in rel_fre_dict.keys():
                rel_fre_dict[line['relation']] = 1
            else:
                rel_fre_dict[line['relation']] += 1
    return rel_fre_dict

trainset_f = get_relation_dict('tacred_train.txt')
print(trainset_f, len(trainset_f))
a = []
for i in trainset_f.keys():
    if i not in remove_rel_list:
        a.append(i)
        
print(a)

a_dict = {}
for it, i in enumerate(a):
    a_dict[i] = it
print(a_dict)
with open('tracred_rel2id_37.json', 'w') as f:
    json.dump(a_dict, f)

{'org:founded': 91, 'org:subsidiaries': 296, 'per:date_of_birth': 63, 'per:cause_of_death': 117, 'per:age': 390, 'per:stateorprovince_of_birth': 38, 'per:countries_of_residence': 445, 'per:country_of_birth': 28, 'per:stateorprovinces_of_residence': 331, 'org:website': 111, 'per:cities_of_residence': 374, 'per:parents': 152, 'per:employee_of': 1524, 'NA': 55112, 'per:city_of_birth': 65, 'org:parents': 286, 'org:political/religious_affiliation': 105, 'per:schools_attended': 149, 'per:country_of_death': 6, 'per:children': 211, 'org:top_members/employees': 1890, 'per:date_of_death': 134, 'org:members': 170, 'org:alternate_names': 808, 'per:religion': 53, 'org:member_of': 122, 'org:city_of_headquarters': 382, 'per:origin': 325, 'org:shareholders': 76, 'per:charges': 72, 'per:title': 2443, 'org:number_of_employees/members': 75, 'org:dissolved': 23, 'org:country_of_headquarters': 468, 'per:alternate_names': 104, 'per:siblings': 165, 'org:stateorprovince_of_headquarters': 229, 'per:spouse': 25

# trash：一些暂时无用代码

In [ ]:
def remove_headRel(old_fp, trainset):
    count = 0
    remove_rel_list = ['NA', 'per:title', 'org:top_members/employees', 'per:employee_of', 'org:alternate_names']
    with open(new_fp, 'w', encoding='utf-8') as f_op:
        with open(old_fp, 'r', encoding = 'utf-8') as f:
                for line in f.readlines():
                    line = json.loads(line) # loads(字符串)， load(文件名字)
                    if line['relation'] not in remove_rel_list:
                        json.dump(line, f_op)
                        f_op.write('\n')
                        count+=1
    return count

In [75]:
def delete_rel(rel_dict, remove_rel_list):
    for key in remove_rel_list:
        del rel_dict[key]



delete_rel(trainset_f, remove_rel_list)
delete_rel(validset_f, remove_rel_list)
delete_rel(testset_f, remove_rel_list)

In [78]:
print(len(trainset_f), len(validset_f), len(testset_f))

37 37 37


In [ ]:
rel_df = pd.DataFrame.from_dict(testset_rel_fre_dict, orient='index')
rel_df.sort_v

In [33]:
def merge_dicts(*dict_args):
    result = {}
    for item in dict_args:
        result.update(item)
    return result
final_dict = merge_dicts(get_relation_dict('tacred_test.txt'),get_relation_dict('tacred_valid.txt'),get_relation_dict('tacred_train.txt'))
rel_pd = pd.DataFrame.from_dict(final_dict,orient='index')

In [51]:
rel_pd1 = pd.DataFrame.from_dict(get_relation_dict('tacred_train.txt'), orient='index')
rel_pd2 = pd.DataFrame.from_dict(get_relation_dict('tacred_valid.txt'), orient='index')
rel_pd3 = pd.DataFrame.from_dict(get_relation_dict('tacred_test.txt'), orient='index')
rel_pd1 = rel_pd1.rename(columns={0:'train'})
rel_pd2 = rel_pd2.rename(columns={0:'valid'})
rel_pd3 = rel_pd3.rename(columns={0:'test'})

In [56]:
rel_pd = rel_pd1.join(rel_pd2).join(rel_pd3)
rel_pd

,train,valid,test
org:founded,91,38,37
org:subsidiaries,296,113,44
per:date_of_birth,63,31,9
per:cause_of_death,117,168,52
per:age,390,243,200
per:stateorprovince_of_birth,38,26,8
per:countries_of_residence,445,226,148
per:country_of_birth,28,20,5
per:stateorprovinces_of_residence,331,72,81
org:website,111,86,26


In [60]:
import numpy as np
import matplotlib as plt
# df = pd.DataFrame(np.random.randn(1000, 4), index=pd.date_range('1/1/2000', periods=1000), columns=list('ABCD'))
rel_df = rel_df.cumsum()
rel_df.plot();

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:


def remove_head_relations(file_path):
    with open(file_path, 'r', encoding = 'utf-8') as f:
        for line in f.readlines():
            line = eval(line)
            

In [61]:
conda install matplotlib

Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... done

## Package Plan ##

  environment location: E:\Anaconda\envs\tf

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cycler-0.10.0              |   py36h009560c_0          13 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    freetype-2.10.2            |       hd328e21_0         470 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    intel-openmp-2020.1        |              216         1.6 MB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    kiwisolver-1.2.0           |   py36h74a9793_0          55 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    matplotlib-3.2.2           |                0          22 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    matplotlib-base-3.2.2    

numpy-base-1.18.5    | 3.8 MB    | #######2   |  73% 
numpy-base-1.18.5    | 3.8 MB    | #######4   |  75% 
numpy-base-1.18.5    | 3.8 MB    | #######6   |  76% 
numpy-base-1.18.5    | 3.8 MB    | #######7   |  78% 
numpy-base-1.18.5    | 3.8 MB    | #######9   |  80% 
numpy-base-1.18.5    | 3.8 MB    | ########1  |  82% 
numpy-base-1.18.5    | 3.8 MB    | ########3  |  83% 
numpy-base-1.18.5    | 3.8 MB    | ########4  |  85% 
numpy-base-1.18.5    | 3.8 MB    | ########6  |  87% 
numpy-base-1.18.5    | 3.8 MB    | ########7  |  88% 
numpy-base-1.18.5    | 3.8 MB    | ########9  |  90% 
numpy-base-1.18.5    | 3.8 MB    | #########1 |  91% 
numpy-base-1.18.5    | 3.8 MB    | #########2 |  92% 
numpy-base-1.18.5    | 3.8 MB    | #########3 |  94% 
numpy-base-1.18.5    | 3.8 MB    | #########6 |  96% 
numpy-base-1.18.5    | 3.8 MB    | #########7 |  98% 
numpy-base-1.18.5    | 3.8 MB    | #########9 |  99% 
numpy-base-1.18.5    | 3.8 MB    | ########## | 100% 

mkl-2020.1           | 99.3